In [25]:
from torch.utils.data import DataLoader

from bert_pytorch.model import ALBERT, ALBERTLM
from bert_pytorch.trainer import BERTTrainer
from bert_pytorch.dataset import ALBERTDataset, WordVocab

import torch
import torch.nn as nn
from torch.utils.data import Dataset

# numpy & pandas
import numpy as np
import pandas as pd

# scikit learn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import textlib as tl



In [27]:
bert = ALBERT(vocab_size=10000, embed_size=128, hidden=256, n_layers=6, attn_heads=8, seq_len=128)
model = ALBERTLM(bert, 128, len(vocab)).to(device)

NameError: name 'self' is not defined

In [6]:
bert = torch.load('../TCL2021_Telco_Embedding_Dataset/albert_model/bert.model.ep0')

In [49]:
x = np.zeros((64,128,256), dtype=np.float32)
x.shape

(64, 128, 256)

In [50]:
fc   = nn.Linear(256, 256)
act  = nn.Tanh()
clsf = nn.Linear(256, 2)      

In [56]:
h = fc(torch.tensor(x))
h.shape
h = act(h)
h.shape
h = clsf(h)
h.shape
h = torch.sum(h, dim=1)
h.shape
h

<ipython-input-56-427790486214>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  h = fc(torch.tensor(x))


tensor([[-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3.8576],
        [-8.8738,  3

In [ ]:
class SOPClassifier(nn.Module):
    def __init__(self, bert: BERT, num_class):
        super().__init__()
        self.bert = bert
        self.decode = nn.Linear(self.bert.hidden, num_class)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self, x, segment_label):
        x = self.bert(x, segment_label)
        out = self.softmax(self.decode(x[:, 0]))
        return out    

In [8]:
x = torch.zeros([64,128], dtype=torch.int32)
s = torch.zeros([64,128], dtype=torch.int32)

decode = nn.Linear(bert.hidden, 37)
softmax = nn.LogSoftmax(dim=-1)

x = bert(x, s)
print( x[:,0].shape )
print( x.shape )


torch.Size([64, 256])
torch.Size([64, 128, 256])


In [ ]:
vocab_path = '../../TCL2021_Telco_Embedding_Dataset/corpora/telco_vocab.dat'
vocab = WordVocab.load_vocab(vocab_path)

In [ ]:
# sop dataset 읽어옴
input_file_name = '../../TCL2021_Telco_Embedding_Dataset/dataset/sop_dataset.xlsx'
try:
    df = pd.read_excel(input_file_name, sheet_name=0, engine='openpyxl')
except FileNotFoundError:
    print(f'{input_file_name}이 없습니다! skip!')

print( df.shape )


In [ ]:
# 첫 모델은 sentence와 label만 써보자
# df_zip = df[ ['sentence', 'label'] ]

y = df.pop('label_clean')
X = df.pop('sentence')

# 문자열로 되어 있는 label을 categorical value로 변환
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
# train / test 분리
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
class SOPDataset(Dataset):
    def __init__(self, X, y, vocab, seq_len):
        self.vocab = vocab
        self.seq_len = seq_len
        self.X = []
        self.y = y
        
        for i, text in enumerate(X):
            try:
                # 클렌징
                cleansed_text = tl.clean_text(text)
            except TypeError:
                print(f'      {i+1} 번째 데이터에 문제가 있어 skip!')
                continue

            # 문장으로 분리하여 배열로 리턴
            sentences = tl.segment_sentences(cleansed_text)
            # 문장 배열을 입릭으로 받아 형태소로 쪼갠 뒤, 다시 하나의 문자열로 변환하여 저장
            corpora = ' '.join(tl.get_corpora(sentences, None, True)).split(' ')
            self.X.append(corpora)            
            
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, item):
        X = self.X[item]
        # token to id
        X = [self.vocab.stoi.get(token, self.vocab.unk_index) for token in X]        
        # [sos] token ids... [eos]
        X = [self.vocab.sos_index] + X + [self.vocab.eos_index]
        
        inp = X[:self.seq_len]
        seg = [1] * self.seq_len
        
        padding = [self.vocab.pad_index for _ in range(self.seq_len - len(inp))]
        inp.extend(padding), seg.extend(padding)
        
        output = {'inp': inp,
                  'seg': seg,
                  'label': y}

        return {key: torch.tensor(value) for key, value in output.items()}        

In [ ]:
train_dataset = SOPDataset(X_train, y_train, vocab, 128)
test_dataset = SOPDataset(X_test, y_test, vocab, 128)

In [ ]:
train_dataset = BERTDataset(args.train_dataset, vocab, 
                              seq_len=args.seq_len)

train_sampler = WeightedRandomSampler(train_dataset.weights, 
                                    train_dataset.data_size)

train_data_loader = DataLoader(train_dataset, 
                               batch_size=args.batch_size, 
                               sampler=train_sampler, 
                               num_workers=args.num_workers, 
                               pin_memory=True)